<a href="https://colab.research.google.com/github/EdgarAgOr/FAP_Cheminformatic_PET/blob/main/2_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q rdkit

! pip install -q molvs

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from molvs.standardize import Standardizer
from molvs.charge import Uncharger, Reionizer
from molvs.fragment import LargestFragmentChooser
from molvs.tautomer import TautomerCanonicalizer
from rdkit.Chem.rdmolops import GetFormalCharge, RemoveStereochemistry

In [ ]:
df = pd.read_csv ("/content/FAP_Bsite_Sin_metales.csv")

In [ ]:
df.columns

In [ ]:
# Vamos a eliminar los compuestos que no tienen reportado un valor numerico de IC50

df2 = df[df.IC50_nM.notna()]

df2

In [ ]:
# Con esto generamos una nueva lista que se llamara 'bioactivity_class
bioactivity_class = []

for i in df2.IC50_nM:
    if float(i) >= 10000000:
        bioactivity_class.append ('Inactive')
    elif float (i) <= 1000000:
        bioactivity_class.append ('Active')
    else:
        bioactivity_class.append ('Intermediate')

In [ ]:
bioactivity_class

In [ ]:
df2.BindingDB_id

In [ ]:
mol_cid = []

for i in df2.BindingDB_id:
    mol_cid.append (i)

In [ ]:
mol_cid

In [ ]:
canonical_smiles = []

for i in df2.Smiles:
    canonical_smiles.append (i)

In [ ]:
standard_value = []

for i in df2.IC50_nM:
    standard_value.append (i)

In [ ]:
selection = ['BindingDB_id', 'Smiles','IC50_nM']

df3 = df2 [selection]

In [ ]:
df3

In [ ]:
# Vamos a anexar a esta tabla la columna de bioactivity

data_tuples = list (zip(mol_cid, canonical_smiles, bioactivity_class, standard_value,DataSource ))

df3 = pd.DataFrame (data_tuples , columns = ['BindingDB_id', 'Smiles', 'bioactivity_class', 'IC50_nM','DataSource' ])

In [ ]:
df3

In [ ]:
# Vamos a eliminar los compuestos que tengan un valor de " Intermedio" en bioactivity_class

df3_class = df3 [df3.bioactivity_class != "Intermediate"]

In [ ]:
df3_class

In [ ]:
# Vamos a salvar este DF "preprocesado"

df3_class.to_csv ("Bioactividad_FAPBSITE_preprocesada_data.csv", index= False)